In [0]:
# Databricks Notebook: Gold (4h Prices × Futures Positions × Fear & Greed) — unified for chart & markers
# - 먼저 gold_fear_greed를 최신화한 뒤 실행하세요. (Silver -> Gold FNG 노트북)
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# ===== (A) 공통 설정 =====
CATALOG = "demo_catalog"
SCHEMA  = "demo_schema"
PRICES  = f"{CATALOG}.{SCHEMA}.gold_prices_4h"            # 4h 가격 + MA + GC/DC
POS_SIL = f"{CATALOG}.{SCHEMA}.silver_futures_positions"  # 리더보드 포지션(Silver)
FNG_GLD = f"{CATALOG}.{SCHEMA}.gold_fear_greed"           # 일단위 Fear & Greed (단일 Gold)
GOLD_J  = f"{CATALOG}.{SCHEMA}.gold_price_positions_4h"   # 3원 조인 집계 테이블
GOLD_X  = f"{CATALOG}.{SCHEMA}.gold_price_positions_4h_exploded"  # 마커(포지션 단건) 뷰
DAYS_BACK = 120           # 최근 N일만 재빌드
FNG_FRESH_DAYS = 3        # FNG 신선도 제한(일). 초과 시 null 처리

spark.sql("SET spark.sql.session.timeZone=UTC")

# ===== (B) 4h 가격 버킷 =====
prices = (
    spark.table(PRICES)
         .where(f"dt >= date_sub(current_date(), {DAYS_BACK})")
         .select(
             "symbol", "bucket_start", "close_4h", "ma50_4h", "ma200_4h",
             "cross_signal", "pct_change_24h", "dt"
         )
         .withColumn("bucket_end", F.col("bucket_start") + F.expr("INTERVAL 4 HOURS"))
)

# ===== (C) Fear & Greed as-of 매핑 (bucket_end 이전 최신 1건) =====
fng = (
    spark.table(FNG_GLD)
         .select(
             F.col("ts_utc").alias("fng_ts"),
             F.col("value").cast("int").alias("fng_value"),
             F.col("value_class").alias("fng_label")
         )
)

prices_fng = (
    prices.alias("p")
      .join(fng.alias("f"), F.col("f.fng_ts") <= F.col("p.bucket_end"), "left")
      .withColumn(
          "rn",
          F.row_number().over(
              Window.partitionBy("p.bucket_start").orderBy(F.col("f.fng_ts").desc())
          )
      )
      .where("rn = 1")
      # 신선도 제한: 오래되면 null
      .withColumn(
          "fng_value",
          F.when(
              F.datediff(F.col("p.bucket_end"), F.col("f.fng_ts")) <= FNG_FRESH_DAYS,
              F.col("f.fng_value")
          ).otherwise(F.lit(None).cast("int"))
      )
      .withColumn(
          "fng_label",
          F.when(
              F.datediff(F.col("p.bucket_end"), F.col("f.fng_ts")) <= FNG_FRESH_DAYS,
              F.col("f.fng_label")
          ).otherwise(F.lit(None).cast("string"))
      )
      .drop("rn", "fng_ts")
)

# ===== (D) 리더보드 포지션 최신값 준비 (uid×symbol별 최신 1건) =====
w_latest = Window.partitionBy("uid", "symbol").orderBy(F.col("event_time").desc())
pos_latest = (
    spark.table(POS_SIL)
         .where(f"dt >= date_sub(current_date(), {DAYS_BACK})")
         .withColumn("rn", F.row_number().over(w_latest))
         .where("rn = 1")
         .drop("rn")
         .select("account_label","uid","symbol","entryPrice","markPrice","pnl","roe","amount","leverage",
                 "event_time","dt")
         .withColumn(
             "side",
             F.when(F.col("amount").isNull(), F.lit(None).cast("string"))
              .when(F.col("amount") >= 0, F.lit("LONG"))
              .otherwise(F.lit("SHORT"))
         )
         .withColumn("notional", F.abs(F.col("amount")) * F.col("markPrice"))
)

# ===== (E) 4h 버킷 × 포지션 매핑 (심볼 동일 & event_time ∈ [start, end)) =====
joined = (
    prices_fng.alias("p")
      .join(
          pos_latest.alias("x"),
          on=[
              F.col("p.symbol") == F.col("x.symbol"),
              F.col("x.event_time") >= F.col("p.bucket_start"),
              F.col("x.event_time") <  F.col("p.bucket_end")
          ],
          how="left"
      )
)

# ===== (F) 버킷 단위 집계 (가격/지표 + FNG + 포지션 요약) =====
bucket_agg = (
    joined.groupBy(
            "p.symbol","p.bucket_start","p.bucket_end","p.dt",
            "p.close_4h","p.ma50_4h","p.ma200_4h","p.cross_signal","p.pct_change_24h",
            "p.fng_value","p.fng_label"
         )
         .agg(
             F.collect_list(
                 F.struct(
                     F.col("x.account_label").alias("account_label"),
                     F.col("x.uid").alias("uid"),
                     F.col("x.side").alias("side"),
                     F.col("x.entryPrice").alias("entryPrice"),
                     F.col("x.markPrice").alias("markPrice"),
                     F.col("x.amount").alias("amount"),
                     F.col("x.notional").alias("notional"),
                     F.col("x.roe").alias("roe"),
                     F.col("x.pnl").alias("pnl"),
                     F.col("x.event_time").alias("position_time")
                 )
             ).alias("positions"),
             F.sum(F.when(F.col("x.side") == "LONG", 1).otherwise(0)).alias("long_cnt"),
             F.sum(F.when(F.col("x.side") == "SHORT",1).otherwise(0)).alias("short_cnt"),
             F.sum(F.when(F.col("x.side") == "LONG", F.col("x.notional")).otherwise(0.0)).alias("long_notional"),
             F.sum(F.when(F.col("x.side") == "SHORT",F.col("x.notional")).otherwise(0.0)).alias("short_notional")
         )
         .selectExpr(
             "symbol","bucket_start","bucket_end","dt",
             "close_4h","ma50_4h","ma200_4h","cross_signal","pct_change_24h",
             "fng_value","fng_label",
             "positions","long_cnt","short_cnt","long_notional","short_notional"
         )
)

# ===== (G) 저장 (집계 테이블) =====
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {GOLD_J} (
  symbol          STRING,
  bucket_start    TIMESTAMP,
  bucket_end      TIMESTAMP,
  close_4h        DOUBLE,
  ma50_4h         DOUBLE,
  ma200_4h        DOUBLE,
  cross_signal    STRING,
  pct_change_24h  DOUBLE,
  fng_value       INT,
  fng_label       STRING,
  positions       ARRAY<STRUCT<
                      account_label:STRING, uid:STRING, side:STRING,
                      entryPrice:DOUBLE, markPrice:DOUBLE, amount:DOUBLE, notional:DOUBLE,
                      roe:DOUBLE, pnl:DOUBLE, position_time:TIMESTAMP
                    >>,
  long_cnt        BIGINT,
  short_cnt       BIGINT,
  long_notional   DOUBLE,
  short_notional  DOUBLE,
  dt              DATE
) USING DELTA
PARTITIONED BY (dt)
""")

bucket_agg.write.mode("overwrite").option("mergeSchema","true").saveAsTable(GOLD_J)
print(f"[JOIN GOLD] refreshed: {GOLD_J}")

# ===== (H) 포지션별(마커) 뷰 =====
exploded = (
    joined
      .select(
          F.col("p.symbol").alias("symbol"),
          F.col("p.bucket_start").alias("bucket_start"),
          F.col("p.bucket_end").alias("bucket_end"),
          F.col("p.close_4h").alias("close_4h"),
          F.col("p.ma50_4h").alias("ma50_4h"),
          F.col("p.ma200_4h").alias("ma200_4h"),
          F.col("p.cross_signal").alias("cross_signal"),
          F.col("p.pct_change_24h").alias("pct_change_24h"),
          F.col("p.fng_value").alias("fng_value"),
          F.col("p.fng_label").alias("fng_label"),
          F.col("x.account_label").alias("account_label"),
          F.col("x.uid").alias("uid"),
          F.col("x.side").alias("side"),
          F.col("x.entryPrice").alias("entryPrice"),
          F.col("x.markPrice").alias("markPrice"),
          F.col("x.amount").alias("amount"),
          F.col("x.notional").alias("notional"),
          F.col("x.roe").alias("roe"),
          F.col("x.pnl").alias("pnl"),
          F.col("x.event_time").alias("position_time"),
          F.col("p.dt").alias("dt")
      )
      .where("uid IS NOT NULL")   # 포지션 없는 버킷은 제외(마커 전용)
)

exploded.createOrReplaceTempView("v_gold_price_positions_4h_exploded")
spark.sql(f"CREATE OR REPLACE TABLE {GOLD_X} AS SELECT * FROM v_gold_price_positions_4h_exploded")
print(f"[JOIN GOLD] exploded markers view: {GOLD_X}")
